# Minggu 1: Pengenalan Sinyal Biomedis
## *Introduction to Biomedical Signals*

**Mata Kuliah:** Pengolahan Sinyal Medis -- Universitas Indonesia

---

### Tujuan Pembelajaran

Setelah menyelesaikan demo ini, mahasiswa diharapkan mampu:

1. Memahami konsep dasar sinyal biomedis (*biomedical signals*) dan konteks klinisnya.
2. Mengenal jenis-jenis sinyal biomedis utama: ECG, EEG, EMG, PPG.
3. Menggunakan Python, NumPy, dan SciPy untuk manipulasi sinyal dasar.
4. Memvisualisasikan sinyal menggunakan Matplotlib dan library `medsinyal`.
5. Menghitung statistik dasar (*basic statistics*) dari sinyal.

## 1. Pendahuluan: Apa itu Sinyal Biomedis?

**Sinyal biomedis** (*biomedical signal*) adalah sinyal listrik, mekanik, atau kimiawi yang dihasilkan oleh tubuh manusia. Sinyal-sinyal ini mengandung informasi penting tentang kondisi fisiologis dan patologis pasien.

### Jenis-Jenis Sinyal Biomedis Utama

| Sinyal | Nama Lengkap | Sumber | Aplikasi Klinis |
|--------|-------------|--------|------------------|
| **ECG** | *Electrocardiogram* | Aktivitas listrik jantung | Deteksi aritmia, infark miokard |
| **EEG** | *Electroencephalogram* | Aktivitas listrik otak | Diagnosis epilepsi, *sleep staging* |
| **EMG** | *Electromyogram* | Aktivitas listrik otot | Evaluasi gangguan neuromuskular |
| **PPG** | *Photoplethysmogram* | Perubahan volume darah | Pengukuran SpO2, denyut nadi |

### Mengapa Pengolahan Sinyal Penting?

Sinyal biomedis mentah (*raw signal*) seringkali terkontaminasi oleh:
- **Noise** dari lingkungan (interferensi jala listrik 50/60 Hz)
- **Artefak** dari gerakan pasien (*motion artifact*)
- **Baseline wander** akibat pernapasan

Pengolahan sinyal digital memungkinkan kita untuk membersihkan, menganalisis, dan mengekstraksi informasi klinis yang bermakna dari sinyal-sinyal tersebut.

## 2. Persiapan: Python / NumPy / SciPy *Refresher*

Mari kita mulai dengan mengimpor library yang diperlukan dan melakukan review singkat operasi dasar.

In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
from scipy import signal as sig

# Library kursus
from medsinyal.io import load_synthetic
from medsinyal.viz import plot_signal, plot_ecg, plot_eeg_bands

print(f"NumPy version : {np.__version__}")
print("Setup selesai!")

In [ ]:
# --- Quick NumPy refresher ---

# Membuat array waktu (time array) dengan np.arange
fs = 500          # frekuensi sampling (Hz)
durasi = 1.0      # durasi (detik)
t = np.arange(0, durasi, 1.0 / fs)
print(f"Jumlah sampel : {len(t)}")
print(f"Durasi         : {t[-1]:.3f} s")

# Membuat sinyal sinusoidal sederhana
f0 = 5  # frekuensi (Hz)
sinyal = np.sin(2 * np.pi * f0 * t)
print(f"Shape sinyal   : {sinyal.shape}")
print(f"Mean           : {np.mean(sinyal):.6f}")
print(f"Std            : {np.std(sinyal):.4f}")

## 3. Visualisasi Sinyal (*Signal Visualization*)

Kita akan memuat data sintetis yang sudah disiapkan dan memvisualisasikannya. Data disimpan dalam format NumPy `.npz`.

In [ ]:
# Memuat data komposisi gelombang sinus
data_sine = load_synthetic("sine_compositions")

print("Keys dalam file:", list(data_sine.keys()))
print(f"Sampling rate  : {float(data_sine['fs'])} Hz")
print(f"Durasi sinyal  : {data_sine['t'][-1]:.1f} s")
print(f"Jumlah sampel  : {len(data_sine['t'])}")

In [ ]:
t_sine = data_sine['t']

# --- Sinyal sinus sederhana ---
plot_signal(t_sine, data_sine['simple_sine'],
            title='Gelombang Sinus Sederhana (5 Hz)',
            ylabel='Amplitudo')
plt.show()

# --- Komposisi multi-frekuensi ---
plot_signal(t_sine, data_sine['multi_freq'],
            title='Komposisi Multi-Frekuensi (1 + 5 + 12 + 30 Hz)',
            ylabel='Amplitudo')
plt.show()

In [ ]:
# --- Perbandingan sinyal bersih vs terkontaminasi noise ---
fig, axes = plt.subplots(2, 1, figsize=(12, 6), sharex=True)

plot_signal(t_sine, data_sine['clean_signal'],
            title='Sinyal Bersih (Clean Signal)', ax=axes[0])
plot_signal(t_sine, data_sine['noisy_signal'],
            title='Sinyal + Interferensi Jala Listrik 50 Hz (Power Line Noise)',
            ax=axes[1])

fig.tight_layout()
plt.show()

print("Perhatikan komponen frekuensi tinggi (50 Hz) yang menumpangi sinyal asli.")
print("Ini adalah contoh umum noise pada rekaman sinyal biomedis.")

## 4. Sinyal ECG (*Electrocardiogram*)

ECG merekam aktivitas listrik jantung melalui elektroda di permukaan kulit. Satu siklus detak jantung menghasilkan pola khas yang disebut **kompleks PQRST**:

- **Gelombang P** -- Depolarisasi atrium (kontraksi serambi)
- **Kompleks QRS** -- Depolarisasi ventrikel (kontraksi bilik), komponen paling dominan
- **Gelombang T** -- Repolarisasi ventrikel (pemulihan bilik)

### Relevansi Klinis

| Parameter | Kondisi Normal | Indikasi Abnormal |
|-----------|---------------|-------------------|
| Interval PR | 0.12 -- 0.20 s | Blok AV jika memanjang |
| Durasi QRS | < 0.12 s | *Bundle branch block* jika melebar |
| Interval QT | 0.36 -- 0.44 s | Risiko aritmia jika memanjang |
| Heart rate | 60 -- 100 bpm | Bradikardia / takikardia |

In [ ]:
# Memuat data ECG sintetis
data_ecg = load_synthetic("synthetic_ecg")

print("Keys:", list(data_ecg.keys()))
print(f"Sampling rate : {float(data_ecg['fs'])} Hz")
print(f"Heart rate    : {float(data_ecg['heart_rate'])} bpm")
print(f"Durasi        : {data_ecg['t'][-1]:.1f} s")
print(f"Jumlah R-peak : {len(data_ecg['r_peak_indices'])}")

In [ ]:
t_ecg = data_ecg['t']
ecg_clean = data_ecg['ecg_clean']
r_peaks = data_ecg['r_peak_indices']

# Visualisasi ECG bersih dengan R-peak markers
plot_ecg(t_ecg, ecg_clean, r_peaks=r_peaks,
         title='Sinyal ECG Sintetis (Clean) dengan R-peak')
plt.show()

In [ ]:
# --- Zoom pada satu siklus PQRST ---
# Mengambil satu beat (sekitar R-peak pertama)
beat_center = r_peaks[1]
fs_ecg = float(data_ecg['fs'])
window = int(0.45 * fs_ecg)  # +/- 0.45 detik
start = beat_center - window
end = beat_center + window

fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(t_ecg[start:end], ecg_clean[start:end], 'k-', linewidth=1.2)
ax.set_title('Satu Siklus ECG -- Kompleks PQRST', fontsize=14)
ax.set_xlabel('Waktu (s)')
ax.set_ylabel('Amplitudo (mV)')
ax.grid(True, alpha=0.3)

# Anotasi komponen PQRST
# Posisi relatif terhadap R-peak (berdasarkan model Gaussian di data generator)
beat_start_time = t_ecg[beat_center] - 0.3  # R-peak di offset 0.3 dalam beat
annotations = {
    'P': (beat_start_time + 0.15, 0.15),
    'Q': (beat_start_time + 0.28, -0.10),
    'R': (beat_start_time + 0.30, 1.00),
    'S': (beat_start_time + 0.32, -0.20),
    'T': (beat_start_time + 0.50, 0.30),
}

for label, (x, y) in annotations.items():
    ax.annotate(label, xy=(x, y), fontsize=14, fontweight='bold',
                color='red', ha='center', va='bottom',
                xytext=(0, 10), textcoords='offset points',
                arrowprops=dict(arrowstyle='->', color='red', lw=1.5))

plt.tight_layout()
plt.show()

print("Gelombang P  : Depolarisasi atrium")
print("Kompleks QRS : Depolarisasi ventrikel (komponen terbesar)")
print("Gelombang T  : Repolarisasi ventrikel")

In [ ]:
# --- Perbandingan ECG bersih vs noisy ---
fig, axes = plt.subplots(2, 1, figsize=(14, 6), sharex=True)

plot_ecg(t_ecg, ecg_clean, title='ECG Bersih (Clean)', ax=axes[0])
plot_ecg(t_ecg, data_ecg['ecg_noisy'],
         title='ECG dengan Noise (Baseline Wander + 50 Hz + Random Noise)',
         ax=axes[1])

fig.tight_layout()
plt.show()

print("Sinyal ECG noisy mengandung:")
print("  - Baseline wander (drift frekuensi rendah akibat pernapasan)")
print("  - Interferensi jala listrik 50 Hz (power line interference)")
print("  - Random noise (noise elektronik)")

## 5. Sinyal EEG (*Electroencephalogram*)

EEG merekam aktivitas listrik otak melalui elektroda di kulit kepala. Sinyal EEG memiliki amplitudo yang sangat kecil (orde mikrovolt, $\mu V$) dan terdiri dari beberapa **pita frekuensi** (*frequency bands*):

| Pita (*Band*) | Rentang Frekuensi | Kondisi Terkait |
|---------------|-------------------|------------------|
| **Delta** ($\delta$) | 0.5 -- 4 Hz | Tidur nyenyak (*deep sleep*) |
| **Theta** ($\theta$) | 4 -- 8 Hz | Mengantuk, meditasi |
| **Alpha** ($\alpha$) | 8 -- 13 Hz | Relaksasi, mata tertutup |
| **Beta** ($\beta$) | 13 -- 30 Hz | Konsentrasi, berpikir aktif |
| **Gamma** ($\gamma$) | 30 -- 50 Hz | Pemrosesan kognitif tingkat tinggi |

### Relevansi Klinis

- **Epilepsi**: Peningkatan aktivitas berlebihan (*spike-and-wave patterns*)
- **Sleep staging**: Perubahan dominansi pita frekuensi selama tahap tidur
- **Brain-Computer Interface (BCI)**: Mengendalikan perangkat melalui sinyal otak

In [ ]:
# Memuat data EEG sintetis
data_eeg = load_synthetic("synthetic_eeg")

print("Keys:", list(data_eeg.keys()))
print(f"Sampling rate : {float(data_eeg['fs'])} Hz")
print(f"Durasi        : {data_eeg['t'][-1]:.1f} s")

In [ ]:
t_eeg = data_eeg['t']

# Visualisasi sinyal EEG gabungan (clean)
plot_signal(t_eeg, data_eeg['eeg_clean'],
            title='Sinyal EEG Sintetis (Gabungan Semua Pita)',
            ylabel='Amplitudo ($\\mu$V)')
plt.show()

print("Perhatikan lonjakan amplitudo pada t=10-15 s (alpha burst, simulasi mata tertutup).")

In [ ]:
# Visualisasi pita frekuensi EEG secara terpisah
bands = {
    'delta': data_eeg['band_delta'],
    'theta': data_eeg['band_theta'],
    'alpha': data_eeg['band_alpha'],
    'beta':  data_eeg['band_beta'],
    'gamma': data_eeg['band_gamma'],
}

plot_eeg_bands(t_eeg, bands,
               title='Pita Frekuensi EEG (EEG Frequency Bands)')
plt.show()

print("Setiap pita frekuensi berkaitan dengan kondisi fisiologis yang berbeda.")
print("Delta: amplitudo besar, frekuensi rendah (tidur nyenyak).")
print("Gamma: amplitudo kecil, frekuensi tinggi (pemrosesan kognitif).")

## 6. Statistik Dasar Sinyal (*Basic Signal Statistics*)

Sebelum melakukan pengolahan lebih lanjut, penting untuk memahami karakteristik dasar sinyal melalui statistik deskriptif:

- **Mean** ($\bar{x}$): Nilai rata-rata sinyal
- **Standard Deviation** ($\sigma$): Ukuran sebaran / variabilitas sinyal
- **Min / Max**: Rentang amplitudo sinyal
- **RMS** (*Root Mean Square*): $\text{RMS} = \sqrt{\frac{1}{N}\sum_{i=1}^{N} x_i^2}$ -- mengukur "kekuatan" sinyal

In [ ]:
def hitung_statistik(sinyal, nama="Sinyal"):
    """Menghitung dan menampilkan statistik dasar suatu sinyal."""
    stats = {
        'Mean': np.mean(sinyal),
        'Std':  np.std(sinyal),
        'Min':  np.min(sinyal),
        'Max':  np.max(sinyal),
        'RMS':  np.sqrt(np.mean(sinyal ** 2)),
    }
    print(f"--- {nama} ---")
    for key, val in stats.items():
        print(f"  {key:5s}: {val:10.4f}")
    return stats


# Perbandingan statistik: ECG clean vs noisy
print("=" * 40)
print("PERBANDINGAN STATISTIK ECG")
print("=" * 40)
stats_clean = hitung_statistik(data_ecg['ecg_clean'], "ECG Clean")
print()
stats_noisy = hitung_statistik(data_ecg['ecg_noisy'], "ECG Noisy")

In [ ]:
# Perbandingan statistik: EEG clean vs noisy vs artifacts
print("=" * 40)
print("PERBANDINGAN STATISTIK EEG")
print("=" * 40)
hitung_statistik(data_eeg['eeg_clean'],          "EEG Clean")
print()
hitung_statistik(data_eeg['eeg_noisy'],          "EEG Noisy")
print()
hitung_statistik(data_eeg['eeg_with_artifacts'], "EEG + Artefak")

print()
print("Perhatikan bahwa:")
print("  - Std dan RMS meningkat dengan adanya noise/artefak.")
print("  - Artefak kedipan mata (eye blink) meningkatkan Max secara signifikan.")
print("  - Statistik dasar bisa menjadi indikator awal kualitas sinyal.")

## 7. Kesimpulan

### Ringkasan Materi Minggu 1

Pada pertemuan ini kita telah mempelajari:

1. **Sinyal biomedis** adalah representasi dari aktivitas fisiologis tubuh (jantung, otak, otot, dll.) yang direkam sebagai sinyal listrik.

2. **Jenis sinyal utama** meliputi ECG (jantung), EEG (otak), EMG (otot), dan PPG (volume darah). Masing-masing memiliki karakteristik frekuensi dan amplitudo yang berbeda.

3. **ECG** menghasilkan pola PQRST yang khas untuk setiap detak jantung. Analisis komponen ini penting untuk diagnosis klinis seperti aritmia dan infark miokard.

4. **EEG** terdiri dari beberapa pita frekuensi (delta, theta, alpha, beta, gamma) yang berkaitan dengan kondisi kesadaran dan aktivitas kognitif.

5. **Noise dan artefak** selalu hadir dalam rekaman sinyal biomedis nyata. Pemahaman statistik dasar membantu kita menilai kualitas sinyal sebelum analisis lebih lanjut.

### Minggu Depan

Kita akan membahas **sampling dan kuantisasi** (*sampling and quantization*) -- bagaimana sinyal analog diubah menjadi sinyal digital, termasuk konsep *Nyquist theorem* dan *aliasing*.